In [ ]:
#@title
#@test {"output": "ignore"}
from __future__ import print_function
import datetime, getpass

print('Last update at %s by %s.' % (datetime.datetime.utcnow(), getpass.getuser()))

Last update at 2021-11-17 05:32:13.284701 by miladg.


In [ ]:
import sys

import pandas as pd
import datetime
import json
import math
import string

import os
import re
import numpy as np
from time import time
from collections import Counter, defaultdict
import itertools
import unicodedata

## Read the input files

In [ ]:
%%shell

fileutil cp -R /cns/ok-d/home/miladg/nlp1/A2-Data /tmp/

E1116 21:32:18.201594  246174 fileutil.cc:1545] couldn't copy `/cns/ok-d/home/miladg/nlp1/A2-Data/1b_benchmark.dev.tokens' to `/tmp/A2-Data/1b_benchmark.dev.tokens': generic::already_exists: destination file /tmp/A2-Data/1b_benchmark.dev.tokens alrea [Line truncated, use colabtools.shell.last_result[0] to inspect.]
E1116 21:32:18.443144  246174 fileutil.cc:1545] couldn't copy `/cns/ok-d/home/miladg/nlp1/A2-Data/1b_benchmark.test.tokens' to `/tmp/A2-Data/1b_benchmark.test.tokens': generic::already_exists: destination file /tmp/A2-Data/1b_benchmark.test.tokens al [Line truncated, use colabtools.shell.last_result[1] to inspect.]
E1116 21:32:18.763171  246174 fileutil.cc:1545] couldn't copy `/cns/ok-d/home/miladg/nlp1/A2-Data/1b_benchmark.train.tokens' to `/tmp/A2-Data/1b_benchmark.train.tokens': generic::already_exists: destination file /tmp/A2-Data/1b_benchmark.train.tokens [Line truncated, use colabtools.shell.last_result[2] to inspect.]
E1116 21:32:19.027701  246174 fileutil.cc:1545] c

In [ ]:
%%shell

ls /tmp/A2-Data

1b_benchmark.dev.tokens
1b_benchmark.test.tokens
1b_benchmark.train.tokens
README.md
subsample_1b_benchmark.sh


## Prepare the tokens

In [ ]:
def unicode_to_ascii(s):
  return ''.join(
    c for c in unicodedata.normalize('NFD', s)
    if unicodedata.category(c) != 'Mn')
  
def normalize_string(s):
  s = unicode_to_ascii(s)
  s = re.sub(r'([!.?])', r' \1', s)
  s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
  s = re.sub(r'\s+', r' ', s)
  return s

def preprocess_sentence(sentence):
  sentence = normalize_string(sentence)
  sentence = sentence.lower().strip()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  # removing contractions
  sentence = re.sub(r"i'm", "i am", sentence)
  sentence = re.sub(r"he's", "he is", sentence)
  sentence = re.sub(r"she's", "she is", sentence)
  sentence = re.sub(r"it's", "it is", sentence)
  sentence = re.sub(r"that's", "that is", sentence)
  sentence = re.sub(r"what's", "that is", sentence)
  sentence = re.sub(r"where's", "where is", sentence)
  sentence = re.sub(r"how's", "how is", sentence)
  sentence = re.sub(r"\'ll", " will", sentence)
  sentence = re.sub(r"\'ve", " have", sentence)
  sentence = re.sub(r"\'re", " are", sentence)
  sentence = re.sub(r"\'d", " would", sentence)
  sentence = re.sub(r"\'re", " are", sentence)
  sentence = re.sub(r"won't", "will not", sentence)
  sentence = re.sub(r"can't", "cannot", sentence)
  sentence = re.sub(r"n't", " not", sentence)
  sentence = re.sub(r"n'", "ng", sentence)
  sentence = re.sub(r"'bout", "about", sentence)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
  sentence = sentence.strip()
  sentence = sentence.translate(str.maketrans('', '', string.punctuation))
  return sentence

In [ ]:
 def tokenize_sentence(lines):
    lines = [preprocess_sentence(i).strip("''").split(" ") for i in lines] 
    print("No of sentences in Corpus: "+str(len(lines)))
    return lines

## Prepare training data

In [ ]:
token_freq = defaultdict(lambda: 0)
with open('/tmp/A2-Data/1b_benchmark.train.tokens', errors='ignore') as file:
  train_lines_raw = file.readlines()
  for l in train_lines_raw:
    l = preprocess_sentence(l)
    for w in l:
      token_freq[w] += 1
train_lines = []
for i in range(len(train_lines_raw)):
  line = "<START> "
  s = preprocess_sentence(train_lines_raw[i])
  for j in range(len(s)):
    if token_freq[s[j]] >= 3:
      line += s[j]
    else:
      print(s[j])
      line += " <UNK>"
  line += " <STOP>"
  train_lines.append(line)

In [ ]:
train_lines_tokenized = tokenize_sentence(train_lines)

No of sentences in Corpus: 61530


In [ ]:
train_lines[1:20]

['<START> long before the advent of e commerce wal mart s founder sam walton set out his vision for a successful retail operation we let folks know we re interested in them and that they re vital to us cause they are he said  <STOP>',
 '<START> a spokesman said the company has been affected by the credit crunch in the united states  <STOP>',
 '<START> abu dhabi is going ahead to build solar city and no pollution city  <STOP>',
 '<START> her back was torn open her liver was ruptured one of her lungs had collapsed and the other was punctured  <STOP>',
 '<START> now it has staging centers where volunteers are coordinating get out the vote efforts said obama s georgia spokeswoman caroline adelman  <STOP>',
 '<START> how about a sibling or family friend  <STOP>',
 '<START> butler s the scorer  <STOP>',
 '<START> in the meantime the multi talented bell gets to showcase her musical chops during the end credits of when in rome when the cast breaks in to a musical dance number  <STOP>',
 '<STAR

## Learn n-gram probabilities

In [ ]:
def freq_of_unique_words(lines):
    bag_of_words = list(itertools.chain.from_iterable(lines)) # change the nested list to one single list
    corpus_word_count = 0 # No of words in the corpus excluding <s> and </s>.
    #count the no. of times a word repeats in the corpus
    count = {}
    for word in bag_of_words:
        if word in count :
            count[word] += 1
        else:
            count[word] = 1
        if word != '<s>' and word != '</s>':
             corpus_word_count +=1
            
    unique_word_count = len(count) - 2 # number of unique words in the corpus excluding <s> and </s>
    
    #print("!!! IT IS EXCLUDING <s> AND </s> !!!")
    print("No of unique words in corpus : "+ str(unique_word_count))
    print("No of words in corpus: "+ str(corpus_word_count))
    
    return count

In [ ]:
unique_word_frequency = freq_of_unique_words(train_lines_tokenized)

No of unique words in corpus : 56710
No of words in corpus: 1497527


In [ ]:
def ngram_freq(lines):
  unigram_frequencies = defaultdict(lambda: 0)
  bigram_frequencies = defaultdict(lambda: 0)
  trigram_frequencies = defaultdict(lambda: 0)
  for sentence in lines:
    # print(sentence)
    prev_prev_word = None
    prev_word = None
    for word in sentence:
      unigram_frequencies[word] += 1
      if prev_word != None and prev_prev_word != None:
        trigram_frequencies[(prev_prev_word, prev_word, word)] += 1
      if prev_word != None:
        bigram_frequencies[(prev_word, word)] += 1
        prev_prev_word = prev_word
      prev_word = word
  return unigram_frequencies, bigram_frequencies, trigram_frequencies

In [ ]:
unigram_frequencies, bigram_frequencies, trigram_frequencies = ngram_freq(train_lines_tokenized)

In [ ]:
unigram_frequencies

defaultdict(<function ngram_freq.<locals>.<lambda> at 0x7fb8e44ed440>, {'start': 61991, 'having': 391, 'a': 33873, 'little': 505, 'flexibility': 16, 'on': 12077, 'that': 13950, 'issue': 311, 'would': 3113, 'go': 649, 'long': 882, 'way': 891, 'to': 37449, 'putting': 117, 'together': 276, 'final': 441, 'package': 124, 'stop': 61756, 'before': 1373, 'the': 83653, 'advent': 7, 'of': 35077, 'e': 258, 'commerce': 56, 'wal': 33, 'mart': 34, 's': 16050, 'founder': 84, 'sam': 36, 'walton': 7, 'set': 693, 'out': 2533, 'his': 6291, 'vision': 64, 'for': 13826, 'successful': 122, 'retail': 107, 'operation': 148, 'we': 3017, 'let': 266, 'folks': 34, 'know': 596, 're': 674, 'interested': 65, 'in': 30194, 'them': 1505, 'and': 33205, 'they': 4595, 'vital': 45, 'us': 1228, 'cause': 207, 'are': 5815, 'he': 8026, 'said': 8893, 'spokesman': 363, 'company': 1181, 'has': 6003, 'been': 3500, 'affected': 94, 'by': 7010, 'credit': 324, 'crunch': 31, 'united': 822, 'states': 792, 'abu': 49, 'dhabi': 21, 'is': 11

In [ ]:
bigram_frequencies

In [ ]:
len(trigram_frequencies)

1085253

In [ ]:
total = 0
for key in unigram_frequencies:
  total += unigram_frequencies.get(key)
total

1497527

In [ ]:
V = 56710
def unigram_prob(w, unigram_frequencies, alpha = 0):
  return (unigram_frequencies.get(w, 0) + alpha) / (total + V * alpha)

def bigram_prob(w1, w2, bigram_frequencies, unigram_frequencies, alpha = 0):
  numerator = bigram_frequencies.get((w1, w2), 0)
  denominator = unigram_frequencies.get(w1, 0)
  return (numerator + alpha) / (denominator + V * alpha)

def trigram_prob(w1, w2, w3, trigram_frequencies,bigram_frequencies, alpha = 0):
  numerator = trigram_frequencies.get((w1, w2, w3), 0)
  denominator = bigram_frequencies.get((w1, w2), 0)
  return (numerator + alpha) / (denominator + V * alpha)

In [ ]:
def compute_perplexity(sentences, unigram_frequencies, bigram_frequencies, trigram_frequencies, alpha=0):
  unigram_perplexity = 0
  bigram_perplexity = 0
  trigram_perplexity = 0

  i = 0
  for sentence in sentences:
    i += 1
    N = len(sentence)
    unigram_product_pi = 1.0
    bigram_product_pi = 1.0
    trigram_product_pi = 1.0

    prev_prev_word = None
    prev_word = None
    for word in sentence:
      unigram_product_pi *= 1/unigram_prob(word, unigram_frequencies, alpha)
      if prev_word != None and prev_prev_word != None:
        trigram_product_pi *= 1/trigram_prob(prev_prev_word, prev_word, word,trigram_frequencies,bigram_frequencies, alpha)
      if prev_word != None:
        bigram_product_pi *= 1/bigram_prob(prev_word, word,bigram_frequencies, unigram_frequencies, alpha)
        prev_prev_word = prev_word
        if prev_prev_word == None: # for the start of the sentence in trigram model
          trigram_product_pi *= 1/bigram_prob(prev_word, word,bigram_frequencies, unigram_frequencies, alpha)
      prev_word = word
    # print(unigram_product_pi, unigram_product_pi**(1.0/N), unigram_perplexity)
    unigram_perp = unigram_product_pi**(1.0/N)
    if not math.isinf(unigram_perp):
      if unigram_perplexity < 1e-7:
        unigram_perplexity = unigram_perp
      else:
        unigram_perplexity = unigram_perplexity / 2.0 + unigram_perp / 2.0
    bigram_perp = bigram_product_pi**(1.0/N)
    if not math.isinf(bigram_perp):
      if bigram_perplexity < 1e-7:
        bigram_perplexity = bigram_perp
      else:
        bigram_perplexity = bigram_perplexity / 2.0 + bigram_perp / 2.0
    trigram_perp = trigram_product_pi**(1.0/N)
    if not math.isinf(trigram_perp):
      if trigram_perplexity < 1e-7:
        trigram_perplexity = trigram_perp
      else:
        trigram_perplexity = trigram_perplexity / 2.0 + trigram_perp / 2.0
  return unigram_perplexity, bigram_perplexity, trigram_perplexity

## Calculate perplexity of training data

In [ ]:
unigram_perplexity, bigram_perplexity, trigram_perplexity = compute_perplexity(train_lines_tokenized,unigram_frequencies, bigram_frequencies, trigram_frequencies, 0.8)

In [ ]:
unigram_perplexity, bigram_perplexity, trigram_perplexity

(917.8073554740781, 1815.8477603106758, 4763.324316403105)

## Read dev data

In [ ]:
with open('/tmp/A2-Data/1b_benchmark.dev.tokens', errors='ignore') as file:
  dev_lines_raw = file.readlines()
  for l in dev_lines_raw:
    l = preprocess_sentence(l)
dev_lines = []
for i in range(len(dev_lines_raw)):
  line = "<START> "
  s = preprocess_sentence(dev_lines_raw[i])
  for j in range(len(s)):
    if token_freq[s[j]] >= 3:
      line += s[j]
    else:
      print(s[j])
      line += " <UNK>"
  line += " <STOP>"
  dev_lines.append(line)
dev_lines_tokenized = tokenize_sentence(dev_lines)

No of sentences in Corpus: 12216


## Calculate perplexity of dev data
We will practice with dev data to improve perplexity

In [ ]:
unigram_perplexity, bigram_perplexity, trigram_perplexity = compute_perplexity(dev_lines_tokenized,unigram_frequencies, bigram_frequencies, trigram_frequencies, 0.8)

In [ ]:
unigram_perplexity, bigram_perplexity, trigram_perplexity

(2341.265244292087, 5538.538611514702, 11931.924321319937)

## Read test data

In [ ]:
with open('/tmp/A2-Data/1b_benchmark.test.tokens', errors='ignore') as file:
  test_lines_raw = file.readlines()
  for l in test_lines_raw:
    l = preprocess_sentence(l)
test_lines = []
for i in range(len(test_lines_raw)):
  line = "<START> "
  s = preprocess_sentence(test_lines_raw[i])
  for j in range(len(s)):
    if token_freq[s[j]] >= 3:
      line += s[j]
    else:
      print(s[j])
      line += " <UNK>"
  line += " <STOP>"
  test_lines.append(line)
test_lines_tokenized = tokenize_sentence(test_lines)

No of sentences in Corpus: 12105


## Calculate perplexity of the test data

In [ ]:
unigram_perplexity, bigram_perplexity, trigram_perplexity = compute_perplexity(test_lines_tokenized,unigram_frequencies, bigram_frequencies, trigram_frequencies, 0.001)

In [ ]:
unigram_perplexity, bigram_perplexity, trigram_perplexity

(3349.1306915224577, 2464.034106349578, 10105.684387365014)

# Choose best alpha per model

In [ ]:
for alpha in [0.0001, 0.001, 0.01, 0.1, 0.3, 0.6, 1, 2, 4, 10]:
  unigram_perplexity, bigram_perplexity, trigram_perplexity = compute_perplexity(dev_lines_tokenized,unigram_frequencies, bigram_frequencies, trigram_frequencies, alpha)
  print(alpha, unigram_perplexity, bigram_perplexity, trigram_perplexity)

0.0001 2828.3214354170173 1984.1173862268754 3336.5418060334005
0.001 2684.460693050114 1384.5772497748421 3281.8319934116125
0.01 2556.768040394101 1476.4919299592261 4523.013616717982
0.1 2441.3284809971897 2524.855823280778 7392.3473872942905
0.3 2387.759864001613 3740.06982163709 9558.722834621763
0.6 2354.365708005539 4926.074741336828 11200.36916396793
1 2331.8543786769724 6067.2798342922615 12514.340756234502
2 2311.9456298632726 8028.914115317099 14367.807804666749
4 2325.2880020441803 10489.285906963787 16199.263225991905
10 2473.7045693545324 14445.63742357371 18399.155677871182
